In [1]:
%reload_ext autoreload
%autoreload 2

In [2]:
import logging

FORMAT = '%(asctime)s %(message)s'

logging.basicConfig(level='INFO', format=FORMAT, datefmt='%d-%b-%y %H:%M:%S')

In [3]:
epoch_key = ("Lotus", 4, 2)

In [27]:
from dask.distributed import Client, TimeoutError, LocalCluster

client = Client(n_workers=32, threads_per_worker=1)
client

Client Scheduler: tcp://127.0.0.1:37637 Dashboard: http://127.0.0.1:32829/status,Cluster Workers: 32 Cores: 32 Memory: 1.62 TB


In [10]:
from src.load_data import load_data

data = load_data(epoch_key, position_to_linearize=['nose_x', 'nose_y'])

19-Oct-20 17:24:53 Loading position info...
19-Oct-20 17:26:05 Loading multiunit...


IndexError: index 20 is out of bounds for axis 1 with size 15

In [13]:
from src.parameters import ANIMALS
from src.load_data import get_interpolated_position_info, make_tetrode_dataframe

position_to_linearize=['tailBase_x', 'tailBase_y']
max_distance_from_well=30
min_distance_traveled=50

position_info = get_interpolated_position_info(
    epoch_key,
    position_to_linearize=position_to_linearize,
    max_distance_from_well=max_distance_from_well,
    min_distance_traveled=min_distance_traveled,
).dropna(subset=["linear_position"])

In [44]:
tetrode_info = make_tetrode_dataframe(
    ANIMALS, epoch_key=epoch_key)
bad_tetrodes = tetrode_info.deadchans.apply(lambda x: x.shape[0] == 4)
tetrode_keys = tetrode_info.loc[tetrode_info.area.isin(
    ['ca1R', 'ca1L']) & ~bad_tetrodes].index

def _time_function(*args, **kwargs):
    return position_info.index

In [46]:
bad_tetrodes

animal  day  epoch  tetrode_number
Lotus   4    2      1                 False
                    2                 False
                    3                 False
                    4                 False
                    5                 False
                    6                 False
                    7                 False
                    8                 False
                    9                 False
                    10                False
                    11                False
                    12                False
                    13                False
                    14                False
                    15                False
                    16                False
                    17                 True
                    18                 True
                    19                 True
                    20                 True
                    21                 True
                    22                 Tr

In [48]:
from src.load_data import get_all_multiunit_indicators

time = position_info.index
multiunits = get_all_multiunit_indicators(
    tetrode_keys, ANIMALS, _time_function)

In [49]:
tetrode_keys

MultiIndex([('Lotus', 4, 2,  1),
            ('Lotus', 4, 2,  2),
            ('Lotus', 4, 2,  3),
            ('Lotus', 4, 2,  4),
            ('Lotus', 4, 2,  5),
            ('Lotus', 4, 2,  6),
            ('Lotus', 4, 2,  7),
            ('Lotus', 4, 2,  8),
            ('Lotus', 4, 2,  9),
            ('Lotus', 4, 2, 10),
            ('Lotus', 4, 2, 11),
            ('Lotus', 4, 2, 12),
            ('Lotus', 4, 2, 14),
            ('Lotus', 4, 2, 15),
            ('Lotus', 4, 2, 16)],
           names=['animal', 'day', 'epoch', 'tetrode_number'])

In [47]:
%debug

> /home/edeno/miniconda3/envs/pose_analysis/lib/python3.6/site-packages/loren_frank_data_processing/multiunit.py(79)get_multiunit_dataframe2()
     77             get_data_filename(animals[animal], day, 'marks'))
     78         multiunit_data = multiunit_file['marks'][0, -1][0, epoch - 1][
---> 79             0, tetrode_number - 1]
     80 
     81         time = pd.TimedeltaIndex(multiunit_data['times'][0, 0].squeeze(),



ipdb>  tetrode_number


16


ipdb>  dir()


['animal', 'animals', 'day', 'epoch', 'multiunit_file', 'tetrode_key', 'tetrode_number']


ipdb>  tetrode_key


('Lotus', 4, 2, 16)


ipdb>  q


In [16]:
tetrode_keys

MultiIndex([('Lotus', 4, 2,  1),
            ('Lotus', 4, 2,  2),
            ('Lotus', 4, 2,  3),
            ('Lotus', 4, 2,  4),
            ('Lotus', 4, 2,  5),
            ('Lotus', 4, 2,  6),
            ('Lotus', 4, 2,  7),
            ('Lotus', 4, 2,  8),
            ('Lotus', 4, 2,  9),
            ('Lotus', 4, 2, 10),
            ('Lotus', 4, 2, 11),
            ('Lotus', 4, 2, 12),
            ('Lotus', 4, 2, 14),
            ('Lotus', 4, 2, 15),
            ('Lotus', 4, 2, 16),
            ('Lotus', 4, 2, 17),
            ('Lotus', 4, 2, 18),
            ('Lotus', 4, 2, 19),
            ('Lotus', 4, 2, 20),
            ('Lotus', 4, 2, 21),
            ('Lotus', 4, 2, 22),
            ('Lotus', 4, 2, 23),
            ('Lotus', 4, 2, 24),
            ('Lotus', 4, 2, 25),
            ('Lotus', 4, 2, 26),
            ('Lotus', 4, 2, 28),
            ('Lotus', 4, 2, 29),
            ('Lotus', 4, 2, 30)],
           names=['animal', 'day', 'epoch', 'tetrode_number'])

In [20]:
!ls /stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2*

/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-01.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-02.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-03.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-04.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-05.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-06.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-07.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-08.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-09.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-10.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-11.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-12.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2-13.mat
/stelmo/abhilasha/animals/Lotus/filterframework/EEG/Lotuseeg04-2

In [25]:
tetrode_info

area     deadchans depth  numcells  \
animal day epoch tetrode_number                                          
Lotus  4   2     1                  ca1L            []    []         0   
                 2                  ca1L            []    []         0   
                 3                  ca1L            []    []         0   
                 4                  ca1L            []    []         0   
                 5                  ca1L            []    []         0   
                 6                  ca1L            []    []         0   
                 7                  ca1L            []    []         0   
                 8                  ca1L            []    []         0   
                 9                  ca1L            []    []         0   
                 10                 ca1L            []    []         0   
                 11                 ca1L            []    []         0   
                 12                 ca1L            []    []         0   
                 13              ca1Lref            []    []         0   
                 14                 ca1L            []    []         0   
                 15                 ca1L            []    []         0   
                 16                 ca1L            []    []         0   
                 17                 ca1R  [1, 2, 3, 4]    []         0   
                 18                 ca1R  [1, 2, 3, 4]    []         0   
                 19                 ca1R  [1, 2, 3, 4]    []         0   
                 20                 ca1R  [1, 2, 3, 4]    []         0   
                 21                 ca1R  [1, 2, 3, 4]    []         0   
                 22                 ca1R  [1, 2, 3, 4]    []         0   
                 23                 ca1R  [1, 2, 3, 4]    []         0   
                 24                 ca1R  [1, 2, 3, 4]    []         0   
                 25                 ca1R  [1, 2, 3, 4]    []         0   
                 26                 ca1R  [1, 2, 3, 4]    []         0   
                 27              ca1Rref  [1, 2, 3, 4]    []         0   
                 28                 ca1R  [1, 2, 3, 4]    []         0   
                 29                 ca1R  [1, 2, 3, 4]    []         0   
                 30                 ca1R  [1, 2, 3, 4]    []         0   
                 31                  NaN  [1, 2, 3, 4]    []         0   
                 32                  NaN  [1, 2, 3, 4]    []         0   

                                suparea  tetrode       tetrode_id  
animal day epoch tetrode_number                                    
Lotus  4   2     1                  hpc        1  Lotus_04_02_001  
                 2                  hpc        2  Lotus_04_02_002  
                 3                  hpc        3  Lotus_04_02_003  
                 4                  hpc        4  Lotus_04_02_004  
                 5                  hpc        5  Lotus_04_02_005  
                 6                  hpc        6  Lotus_04_02_006  
                 7                  hpc        7  Lotus_04_02_007  
                 8                  hpc        8  Lotus_04_02_008  
                 9                  hpc        9  Lotus_04_02_009  
                 10                 hpc       10  Lotus_04_02_010  
                 11                 hpc       11  Lotus_04_02_011  
                 12                 hpc       12  Lotus_04_02_012  
                 13                  cc       13  Lotus_04_02_013  
                 14                 hpc       14  Lotus_04_02_014  
                 15                 hpc       15  Lotus_04_02_015  
                 16                 hpc       16  Lotus_04_02_016  
                 17                 hpc       17  Lotus_04_02_017  
                 18                 hpc       18  Lotus_04_02_018  
                 19                 hpc       19  Lotus_04_02_019  
                 20                 hpc       20  Lotus_04_02_020  
                 21    

animal  day  epoch  tetrode_number
Lotus   4    2      1                 False
                    2                 False
                    3                 False
                    4                 False
                    5                 False
                    6                 False
                    7                 False
                    8                 False
                    9                 False
                    10                False
                    11                False
                    12                False
                    13                False
                    14                False
                    15                False
                    16                False
                    17                 True
                    18                 True
                    19                 True
                    20                 True
                    21                 True
                    22                 Tr

In [50]:
client.close()